In [1]:
import os
import pickle

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit

from tasks import flux

## Flux

In [9]:
path_ds = os.path.join('/Users','thiago','Neurodata','Datasets','flux_dsB1/')
cal = ('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')
listSubj = np.array(os.listdir(path_ds))
listSubj = listSubj[np.logical_not([n.startswith('.') for n in listSubj])]
listSubj = listSubj[[os.path.isdir(os.path.join(path_ds,n)) for n in listSubj]]
listSubj.sort()
listSubj

array(['TG023', 'TG024', 'TG025', 'TG026'], dtype='<U10')

In [56]:
dicVI = {'armNo':[],'interval':[],'subj':[], 'date':[],'ndx':[]}

ndxOffset=0
for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    if not os.path.isdir(os.path.join(path_ds,nameSubj)):
        continue
    listSess = np.array(os.listdir(os.path.join(path_ds,nameSubj)))
    listSess = listSess[[n.endswith('b.pkl') for n in listSess]]
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]

        with open(os.path.join(path_ds,nameSubj,nameSess), 'rb') as filehandler:
            sess = pickle.load(filehandler)

        sessname = sess.fname.split('_')
        
        for iArm in set(sess.dfRwd['arm']):
            dfRwd = sess.dfRwd.loc[sess.dfRwd['arm']==iArm]
            dfSetup = sess.dfSetup.loc[sess.dfSetup['arm']==iArm]
            ndx=dfRwd.index
            for iRew in range(len(ndx)-1):
                tsRwd=dfRwd['tsRwd'].loc[ndx[iRew]]                
                dicVI['armNo'].append(iArm)
                dicVI['interval'].append(dfSetup['tsSetup'].loc[dfSetup['tsSetup']>tsRwd].min()-tsRwd)
                dicVI['subj'].append(nameSubj)
                dicVI['date'].append(sessname[3] + str(np.where([n == sessname[2][0:3] for n in cal])[0][0]+1).zfill(2) + sessname[2][3:6])
                dicVI['ndx'].append(ndx[iRew]+ndxOffset)
        ndxOffset=max(dicVI['ndx'])+1
        

dfVI = pd.DataFrame(dicVI)
dfVI=dfVI.set_index('ndx')
dfVI=dfVI.sort_index()
dfVI=dfVI[['subj', 'date','armNo','interval']]

In [69]:
dfVI['armVal']=np.nan



for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    ndxSubj=dfVI['subj']==nameSubj
    listSess = np.array(list(set(dfVI[ndxSubj].date)))
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]
        ndxSess=np.logical_and(ndxSubj,dfVI['date']==nameSess)
        mu=np.full(3,np.nan)
        sigma=np.full(3,np.nan)
        armVal=np.full(3,np.nan)
        for iArm in set(dfVI[ndxSess]['armNo']):
            ndxArm=np.logical_and(ndxSess,dfVI['armNo']==iArm)
            mu[iArm]=dfVI[ndxArm]['interval'].mean()
            sigma[iArm]=dfVI[ndxArm]['interval'].std()
        for iArm in set(dfVI[ndxSess]['armNo']):
            try:
                assert(np.arange(3)[mu[iArm]==sorted(mu)].item()==np.arange(3)[sigma[iArm]==sorted(sigma)].item())
            except:
                print(nameSubj,nameSess)
            
            ndxArm=np.logical_and(ndxSess,dfVI['armNo']==iArm)
            dfVI.iloc[np.arange(len(dfVI))[ndxArm],np.arange(len(dfVI.columns))[dfVI.columns=='armVal'].item()]=np.arange(3)[mu[iArm]==sorted(mu)].item()

dfVI.armVal=dfVI.armVal.astype(int)

TG026 20180625
TG026 20180625


In [71]:
with open(os.path.join('/Users','thiago','Neurocode','Scripts_TaskSuite','3AF_report','dfVI.pkl'),'wb') as fhandle:
    pickle.dump(dfVI,fhandle)

In [11]:
dicLing = {'armNo':[],'armVal':[],'lingert':[],'nRew':[],'subj':[], 'date':[]}

for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    if not os.path.isdir(os.path.join(path_ds,nameSubj)):
        continue
    listSess = np.array(os.listdir(os.path.join(path_ds,nameSubj)))
    listSess = listSess[[n.endswith('b.pkl') for n in listSess]]
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]

        with open(os.path.join(path_ds,nameSubj,nameSess), 'rb') as filehandler:
            sess = pickle.load(filehandler)

        sessname = sess.fname.split('_')
        tsPoke = sess.dfPokes['tsPoke']
        arm = sess.dfPokes['arm']
        
        MeanABC=sess.params.index[[n.startswith('Mean') for n in sess.params.index]]
        ndxSwi=np.where(sess.dfPokes['isSwitch'])[0]
        
        for i in range(len(ndxSwi)-1):
            assert(arm.iloc[ndxSwi[i+1]-1]==arm.iloc[ndxSwi[i]])
            dicLing['armNo'].append(arm.iloc[ndxSwi[i]])
            dicLing['armVal'].append(sess.params[MeanABC][arm.iloc[ndxSwi[i]]])
            dicLing['lingert'].append(tsPoke.iloc[ndxSwi[i+1]-1]-tsPoke.iloc[ndxSwi[i]])
            dicLing['subj'].append(nameSubj)
            dicLing['date'].append(sessname[3] + str(np.where([n == sessname[2][0:3] for n in cal])[0][0]+1).zfill(2) + sessname[2][3:6])
            dicLing['nRew'].append(sess.dfPokes['isRwded'].iloc[ndxSwi[i]:ndxSwi[i+1]].sum())
            

dfLing = pd.DataFrame(dicLing)
dfLing=dfLing[['subj', 'date','armNo','armVal','nRew','lingert']]
dfLing=dfLing.sort_values(['subj','date'])

In [13]:
with open(os.path.join('/Users','thiago','Neurocode','Scripts_TaskSuite','3AF_report','dfLing.pkl'),'wb') as fhandle:
    pickle.dump(dfLing,fhandle)